<a href="https://colab.research.google.com/github/Shreyash007/deep_learning_course/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

Import libraries

In [5]:
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import numpy as np
import pdb
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import wandb

Labelling dataset

In [10]:
labels = {0: "T-shirt", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle boot"}
lab = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
(X_train, Y_train), (x_test, y_test) = fashion_mnist.load_data()
indexes = [0,1,3,5,6,8,16,18,19,23]
images = [X_train[i] for i in indexes]
titles = ['Ankle Boot', 'T-Shirt', 'Dress', 'Pullover', 'Sneaker', 'Sandal', 'Trouser', 'Shirt', 'Coat', 'Bag']
label = 9
wandb.init(project='CS-6910 A1',entity='shreyashgadgil007')
wandb.log({"Fashion-MNIST images": wandb.Image(images[label], caption=titles[label])})
wandb.log({"Label": label})

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
